# E91 QKD

In [ ]:
# Name: Brian Bahk
# Name: Minjae Cho
# Name: Elijah Olive

In [ ]:
# Install qiskit
!pip install qiskit

In [ ]:
#@title #Sample to test / Reference; no need to run
from qiskit import QuantumCircuit
import numpy as np
import random
# Create a quantum circuit with two qubits
qc = QuantumCircuit(2, 2)

# Create a Psi- state
qc.h(1)
qc.cx(1, 0)
qc.x(1)
qc.z(1)

# Add a measurement to both qubits
qc.measure([0, 1], [0, 1])
print(qc)

# Execute using the Sampler primitive
from qiskit.primitives.sampler import Sampler
sampler = Sampler()
job = sampler.run(qc, shots=1024)
result = job.result()
print(f" > Quasi probability distribution: {result.quasi_dists}")

# Get the only output (if 1 shot)
print(str(result.quasi_dists[0])[1])

# Get the probability of (result)
print(result.quasi_dists[0][2])

          ┌───┐     ┌─┐        
q_0: ─────┤ X ├─────┤M├────────
     ┌───┐└─┬─┘┌───┐└╥┘┌───┐┌─┐
q_1: ┤ H ├──■──┤ X ├─╫─┤ Z ├┤M├
     └───┘     └───┘ ║ └───┘└╥┘
c: 2/════════════════╩═══════╩═
                     0       1 
 > Quasi probability distribution: [{1: 0.4775390625, 2: 0.5224609375}]
1
0.5224609375


In [ ]:
#@title #Photon / Reference; no need to run
#This cell's contents should match those of photon.py

import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class Photon:

    def __init__(self, Hcomp=0, Vcomp=0):
        self.alpha = Hcomp
        self.beta  = Vcomp

    # This is for debugging purposes only!
    def toString(self):
        if np.isreal(self.alpha):
            string = str(self.alpha) + "|H> "
        else:
            string = str(self.alpha) + "|H> "
        if np.isreal(self.beta):
            if self.beta >= 0:
                string += "+ " + str(self.beta) + "|V>"
            else:
                string += "- " + str(-self.beta) + "|V>"
        else:
            string += "+ " + str(self.beta) + "|V>"
        return string

    def prepareVacuum(self):
        energyPerMode = 0.5; # in units of hbar*omega
        x0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        self.alpha = complex(x0, y0)
        self.beta  = complex(x1, y1)

    def prepare(self, alpha, beta, avgPhotonNumber):
        if avgPhotonNumber < 0:
            raise InputError()
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = alpha * np.sqrt(avgPhotonNumber) + vac.alpha
        self.beta  = beta  * np.sqrt(avgPhotonNumber) + vac.beta

    def prepareH(self, avgPhotonNumber):
        self.prepare(1, 0, avgPhotonNumber)

    def prepareV(self, avgPhotonNumber):
        self.prepare(0,1, avgPhotonNumber)

    def prepareD(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1/np.sqrt(2), avgPhotonNumber)

    def prepareA(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1/np.sqrt(2), avgPhotonNumber)

    def prepareR(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1j/np.sqrt(2), avgPhotonNumber)

    def prepareL(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1j/np.sqrt(2), avgPhotonNumber)

    def measureHV(self, probDarkCount):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.alpha)**2
        intensityV = abs(self.beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

    def measureDA(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a+b)/np.sqrt(2)
        self.beta  = (a-b)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def measureRL(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a-b*1j)/np.sqrt(2)
        self.beta  = (a+b*1j)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "R"
        if outcome == "V": return "L"
        else: return outcome

    def applyPolarizer(self, theta, phi):
	# Apply a polarizing filter according to the input parameters.
	# theta=0,    phi=0:     H polarizer
	# theta=pi/2, phi=0:     V polarizer
	# theta=pi/4, phi=0:     D polarizer
	# theta=pi/4, phi=pi:    A polarizer
	# theta=pi/4, phi=+pi/2: R polarizer
	# theta=pi/4, phi=-pi/2: L polarizer
        z = np.exp(1j*phi)
        a = self.alpha
        b = self.beta
        self.alpha = a*(1+np.cos(2*theta))/2 + b*np.sin(2*theta)/2*np.conj(z)
        self.beta  = a*np.sin(2*theta)/2*z + b*(1-np.cos(2*theta))/2
        # Now add an extra vacuum component.
        vac = Photon()
        vac.prepareVacuum()
        a = vac.alpha
        b = vac.beta
        self.alpha = self.alpha + a*np.sin(theta)**2 + b*(-np.sin(2*theta)/2)*np.conj(z)
        self.beta  = self.beta  + a*(-np.sin(2*theta)/2)*z + b*np.cos(theta)**2

    def applyUnitaryGate(self, theta, phi, lamb):
        U = [[0,0],[0,0]]
        z1 = np.exp(1j*phi)
        z2 = -np.exp(1j*lamb)
        z3 = np.exp(1j*(lamb+phi))
        U[0][0] = np.cos(theta/2)
        U[1][0] = np.sin(theta/2)*z1
        U[0][1] = np.sin(theta/2)*z2
        U[1][1] = np.cos(theta/2)*z3
        a = self.alpha
        b = self.beta
        self.alpha = U[0][0]*a + U[0][1]*b
        self.beta  = U[1][0]*a + U[1][1]*b

    def applyXGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(np.pi, 0, np.pi)

    def applyYGate(self):
        # Applies the Pauli Y gate
        self.applyUnitaryGate(np.pi, np.pi/2, np.pi/2)

    def applyZGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(0, np.pi, 0)

    def applyHGate(self):
        # Applied the Hadamard (half-wavelength) gate
        self.applyUnitaryGate(np.pi/2, 0, np.pi)

    def applyQGate(self):
        # Applies the SH (quarter-wavelength) gate
        self.applyUnitaryGate(np.pi/2, np.pi/2, np.pi)

    def applyNoisyGate(self, p):
        # This operation acts as a depolarizing channel.
	# p = 0 leaves the photon unchanged.
	# p = 1 yields a completely random photon.
	# 0 < p < 1 yields a partially random photon.
        if p < 0 or p > 1:
            raise InputError
        theta = np.arccos(1 - 2*random.uniform(0,1)*p)
        phi   = p*(2*random.uniform(0,1) - 1)*np.pi
        lamb  = p*(2*random.uniform(0,1) - 1)*np.pi
        self.applyUnitaryGate(theta, phi, lamb)

    def applyAttenuation(self, r):
	# This operation acts as a partially reflecting beam splitter.
	# r = 0 leaves the photon unchanged.
	# r = 1 completely absorbs the photon, leaving a vacuum state.
	# 0 < r < 1 partially attenuates the photon and adds some vacuum.
	# r is the reflectivity.
        if r < 0 or r > 1:
            raise InputError
        t = np.sqrt(1-r*r) # t is the transmissivity.
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = (self.alpha)*t + (vac.alpha)*r
        self.beta  = (self.beta )*t + (vac.beta)*r

In [ ]:
#@title #E91? / Reference; no need to run

import numpy as np
import random

class E91Simulation:
    def __init__(self, num_pairs):
        self.num_pairs = num_pairs
        self.Alice_photons = []
        self.Bob_photons = []
        self.create_entangled_pairs()

    def create_entangled_pairs(self):
        """Create entangled pairs (Bell state |Ψ⁻⟩ = (|HV⟩ - |VH⟩)/sqrt(2)) for Alice and Bob."""
        for _ in range(self.num_pairs):
            alice_photon = Photon()
            bob_photon = Photon()
            # Entangle the photons
            alice_photon.prepareH(1)  # Prepare Alice's photon in |H>
            bob_photon.prepareV(1)    # Prepare Bob's photon in |V>
            self.Alice_photons.append(alice_photon)
            self.Bob_photons.append(bob_photon)

    def measure_photons(self):
        """Both Alice and Bob measure photons in randomly chosen bases."""
        bases = ['HV', 'DA', 'RL']  # Define measurement bases: HV, Diagonal-Antidiagonal, Right-Left circular
        results_alice = []
        results_bob = []
        for i in range(self.num_pairs):
            alice_base = random.choice(bases)
            bob_base = random.choice(bases)
            if alice_base == 'HV':
                alice_result = self.Alice_photons[i].measureHV(0)
            elif alice_base == 'DA':
                alice_result = self.Alice_photons[i].measureDA(0)
            elif alice_base == 'RL':
                alice_result = self.Alice_photons[i].measureRL(0)

            if bob_base == 'HV':
                bob_result = self.Bob_photons[i].measureHV(0)
            elif bob_base == 'DA':
                bob_result = self.Bob_photons[i].measureDA(0)
            elif bob_base == 'RL':
                bob_result = self.Bob_photons[i].measureRL(0)

            results_alice.append((alice_base, alice_result))
            results_bob.append((bob_base, bob_result))

        return results_alice, results_bob

    def sift_keys(self, results_alice, results_bob):
        """Sift keys to find matching measurements and discard others."""
        alice_key = []
        bob_key = []
        for a, b in zip(results_alice, results_bob):
            if a[0] == b[0] and a[1] in ['H', 'V', 'D', 'A', 'R', 'L'] and b[1] in ['H', 'V', 'D', 'A', 'R', 'L']:
                alice_key.append(a[1])
                bob_key.append(b[1])
        return alice_key, bob_key

    def measureDA(self, probDarkCount):
      """ Measure in Diagonal (|D⟩) and Antidiagonal (|A⟩) basis. """
      # Transform to diagonal basis
      new_alpha = (self.alpha + self.beta) / np.sqrt(2)
      new_beta = (self.alpha - self.beta) / np.sqrt(2)

      # Store transformed state back to the photon
      self.alpha = new_alpha
      self.beta = new_beta

      # Measure in HV basis after transformation
      result = self.measureHV(probDarkCount)
      if result == "H":
        return "D"  # Detected in |D⟩
      elif result == "V":
        return "A"  # Detected in |A⟩
      else:
        return result  # No detection or invalid

    def measureRL(self, probDarkCount):
      """ Measure in Right (|R⟩) and Left (|L⟩) circular polarization basis. """
      # Transform to circular polarization basis
      new_alpha = (self.alpha + 1j * self.beta) / np.sqrt(2)
      new_beta = (self.alpha - 1j * self.beta) / np.sqrt(2)

      # Store transformed state back to the photon
      self.alpha = new_alpha
      self.beta = new_beta

      # Measure in HV basis after transformation
      result = self.measureHV(probDarkCount)
      if result == "H":
        return "R"  # Detected in |R⟩ (Right-hand circular polarization)
      elif result == "V":
        return "L"  # Detected in |L⟩ (Left-hand circular polarization)
      else:
        return result  # No detection or invalid

    def measureHV(self, probDarkCount):
      """ Measure in Horizontal (|H⟩) and Vertical (|V⟩) basis. """
      threshold = -0.5 * np.log(1 - np.sqrt(1 - probDarkCount))
      intensityH = abs(self.alpha) ** 2
      intensityV = abs(self.beta) ** 2
      # The photon is absorbed by the detector:
      self.prepareVacuum()
      # The outcome is determined by threshold exceedances:
      if intensityH <= threshold and intensityV <= threshold:
        return "N"  # no detection (invalid measurement)
      elif intensityH > threshold and intensityV <= threshold:
        return "H"  # single H photon detected
      elif intensityH <= threshold and intensityV > threshold:
        return "V"  # single V photon detected
      else:
        return "M"  # multiple detections (invalid measurement)



# Example usage:
e91_sim = E91Simulation(10)  # Simulate E91 with 10 photon pairs
results_alice, results_bob = e91_sim.measure_photons()
alice_key, bob_key = e91_sim.sift_keys(results_alice, results_bob)
print("Alice's key:", alice_key)
print("Bob's key:", bob_key)


Alice's key: []
Bob's key: []


<ipython-input-267-3d4801e54a84>:71: RuntimeWarning: divide by zero encountered in log
  threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))


In [ ]:
#@title #QC

from qiskit import QuantumCircuit
from qiskit.primitives.sampler import Sampler
import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class QC:

    def __init__(self):
        self.qc = QuantumCircuit(2, 2)

        # Create a Psi- state
        self.qc.h(1)
        self.qc.cx(1, 0)
        self.qc.x(1)
        self.qc.z(1)

        self.alice_alpha = 0
        self.alice_beta = 0
        self.bob_alpha = 0
        self.bob_beta = 0

    def prepareVacuum(self):
        energyPerMode = 0.5; # in units of hbar*omega
        x0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x2 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y2 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x3 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y3 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        self.alice_alpha = complex(x0, y0)
        self.alice_beta  = complex(x1, y1)
        self.bob_alpha = complex(x2, y2)
        self.bob_beta  = complex(x3, y3)

    def prepareHV(self):
        vac = QC()
        vac.prepareVacuum

        # Add a measurement to both qubits
        self.qc.measure([0, 1], [0, 1])

        # Execute using the Sampler primitive
        sampler = Sampler()
        job = sampler.run(self.qc, shots=1)
        result = job.result()
        ab = int(str(result.quasi_dists[0])[1])
        if ab == 1:
            self.alice_alpha = 1 + vac.alice_alpha
            self.alice_beta  = 0 + vac.alice_beta
            self.bob_alpha = 0 + vac.bob_alpha
            self.bob_beta  = 1 + vac.bob_beta
        elif ab == 2:
            self.alice_alpha = 0 + vac.alice_alpha
            self.alice_beta  = 1 + vac.alice_beta
            self.bob_alpha = 1 + vac.bob_alpha
            self.bob_beta  = 0 + vac.bob_beta

    def prepareDA(self):
        vac = QC()
        vac.prepareVacuum

        # Put hadamard before measuring to measure in D/A basis
        self.qc.h(0)
        self.qc.h(1)

        # Add a measurement to both qubits
        self.qc.measure([0, 1], [0, 1])

        # Execute using the Sampler primitive
        sampler = Sampler()
        job = sampler.run(self.qc, shots=1)
        result = job.result()
        ab = int(str(result.quasi_dists[0])[1])
        # Psi- = 1/2sqrt2 * (|AD> - |DA>)
        if ab == 1: # 01; DA
            self.alice_alpha = 1/np.sqrt(2) + vac.alice_alpha
            self.alice_beta  = 1/np.sqrt(2) + vac.alice_beta
            self.bob_alpha = 1/np.sqrt(2) + vac.bob_alpha
            self.bob_beta  = -1/np.sqrt(2) + vac.bob_beta
        elif ab == 2: # 10; AD
            self.alice_alpha = 1/np.sqrt(2) + vac.alice_alpha
            self.alice_beta  = -1/np.sqrt(2) + vac.alice_beta
            self.bob_alpha = 1/np.sqrt(2) + vac.bob_alpha
            self.bob_beta  = 1/np.sqrt(2) + vac.bob_beta

    # Call after preparing (aka measuring one of the qubits)
    def getAliceKeyHV(self, probDarkCount):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.alice_alpha)**2
        intensityV = abs(self.alice_beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

    def getAliceKeyDA(self, probDarkCount):
        a = self.alice_alpha
        b = self.alice_beta
        self.alice_alpha = (a+b)/np.sqrt(2)
        self.alice_beta  = (a-b)/np.sqrt(2)
        outcome = self.getAliceKeyHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def measureHV(self, probDarkCount):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.bob_alpha)**2
        intensityV = abs(self.bob_beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

    def measureDA(self, probDarkCount):
        a = self.bob_alpha
        b = self.bob_beta
        self.bob_alpha = (a+b)/np.sqrt(2)
        self.bob_beta  = (a-b)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def eveSendH(self):
        self.bob_alpha = 1
        self.bob_beta  = 0

    def eveSendV(self):
        self.bob_alpha = 0
        self.bob_beta  = 1

    def eveSendD(self):
        self.bob_alpha = 1/np.sqrt(2)
        self.bob_beta  = 1/np.sqrt(2)

    def eveSendA(self):
        self.bob_alpha = 1/np.sqrt(2)
        self.bob_beta  = -1/np.sqrt(2)

In [ ]:
n = 1000 # number of photons

In [ ]:
# Alice chooses the encoding basis for each key bit.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisAlice = ""
for index in range(n):
  if random.randint(0,1) == 0:
    basisAlice += "+"
  else:
    basisAlice += "x"
print("basisAlice  = " + basisAlice)

basisAlice  = xxx++++xxx+x++xx+++xx++x++xx++xx+++xx+xx+++xx+xx++x+xx+xx++x+x+xxxxx++xxx+++++++x++xxx++xxx++xx+x++xxxx++x++xx+xx+++xx++xxxx+x+x+x+xxx++x+xx+++xxx+++xxx++xx+xx++++++x+x+xxx+x+++x+xxx++xx++xx+xx+x++x+++xxxxx+xx+xxx+x++++xx+++xxx++x+xxx+x++x+x+x+xx+++++x+xxxx++x+xxxx+x++x+++++++xxx++xx++xx++x+++x+xx+++x+xxxx++++x+x+x++xxx+++++xxx++++xx++x+xxx+xxxxx++++x+x+xxxx+xxx+++xx+x+xx+x+++xxxx+x++xx++xx++++xxxxx++x+x+xx++x+++xx+x+xxxxx++x+x+x++++++x++xx+x+xx++xxx++x+x+xx+xx+x+xxxx+++xx+xx+xx+++++xx+x++xxxx+++++++++xxx+++xxxxx++x+xxx+++x+xx++xx+++x++xx+xx+x+x+++++++xxxxx++++xx+x++xxx++x++++x+++x+++x+xx++xxxxx++xxx+xx++++x+x+x+xx+x++x+x+x++xxx+x++x+++++xx++xx+x+x+xx++x+++xxx+x+xxxxx+xxx+xx++xx+x++++++x++x+++x++x+++xxx+xx++x+x+xxxx+xx++xxx++++++xx+x+++++x++++x++x++x+x+xxx+++x+++x++++x+x+xx+xx+x++xxxxxxx++x+x+x+++x+x+xxxxxx+xx+x+x++x+xxxxxx+xx++++xxx+x++x++x++x+xx+++x+x+++x++++x++x+xxx++xx+x+x++x++xxxx+x+x+x+++xxxx+x++x++xx+x+x+xxx++++x++x+x++++++x++xxxxxx+x+x+x+xxx+xx+x++xx++++xxx+xx+++x

In [ ]:
# Alice measures Bell State based on chosen basis
qcArray = [QC() for i in range(n)]
for i in range(n):
  if basisAlice[i] == "+":
    QC.prepareHV(qcArray[i])
  elif basisAlice[i] == "x":
    QC.prepareDA(qcArray[i])

In [ ]:
# Alice gets the raw key.
polarizationAlice = ""
for i in range(n):
  if basisAlice[i] == "+":
    polarizationAlice += QC.getAliceKeyHV(qcArray[i], .5)
  elif basisAlice[i] == "x":
    polarizationAlice += QC.getAliceKeyDA(qcArray[i], .5)
print("keyAlice  = " + polarizationAlice)

keyAlice  = ADAVVHVDDAHDVHDDVHHDDVVAHHAAVHDDHVHAAVDDVHVAAHAAHHDVDDVADVVAHAHAAAAAVHDDDHVHVVHHDVHAADVHADDVHADHAHHDDADVVAVHADVDAVHVADVHDDAAVAHAVAVAAAVVAHDDHHHAADHHHDADVVADVADHHHHHHAVDVADDVDVVHAVDDAHVDAHHDAVADHDHHDVHVDDADAVADVDADHAHVVHADHVHDDAHHAVDADHAVVAVDHDVDDHHHVVDVAADDHVAVADADVAVVDHHVHVVHDAAVHAAVHDAHHDVHVDHDDHVVDVADAAHVHHDHAVAHHADAVVHVHAADHVHHADVHDHADAVADAAAVHHVAHAVAADAHDDAVVVDAHDVAAVDHHHDADDVAHHADHVDDVHVHAADDDHVAVDHADVHAVHVDDHDHADADAHHDHDHAVHVVHVAHHADHAVAAHVDDAVHAVDHAAHDDVDVDDDDVVHAAVADVAAVHVVHDAHAHHDDAAHHVHHVVVVDADVVVDDDAAHHAVDDDVHVDHADHVDAVVVDHVDAVDDVAVAVVVVHVHDDAADHHVVADHDVVDDDHVDVHVVDHVVDVHHDVDAVHDDDDDVVDAAHADHVVVAHAVAVDDHDHVAHDVDVHDDDHAVHAVHVVHAAHVDAHDVDVAAHHDVHVDDDHDVDDDAAVADAVAAVHDDHAVHHHVHAVVDHVVAHHDHHVDAAHAAHVDVDVAADDHDDVVDDAHVHHHHAAHDHVVVHDVVHVAVHDHHDHAHDDAVVVAVHVDVVVHDVAVDDHAAVAVHDDDADDDHVAVDHAVHVDHAHDADDDAHDAHDVAHHAVADAAADVDAHHVHAADHAHVAHHDHHDHDDHHHDHAHVHDVHHVDVVDHADDVVADHAVDVVAHHDDDDVDVDHDHHVDAADVDHVAHVDDHDVDHAAAVVHVDHVDHDHHVVVVDHHDAAADDHAVAHDVAAAHAAVAVHDDHVHVDAAVDAHHHDDV

In [ ]:
# Alice infers the raw key.
# keyAlice should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement
keyAlice = ""
# TODO: Put your code here.
for i in range(n):
    if basisAlice[i] == '+':
        if polarizationAlice[i] == 'H':
            keyAlice += '0'
        elif polarizationAlice[i] == 'V':
            keyAlice += '1'
        else:
            keyAlice += '-'
    elif basisAlice[i] == 'x':
        if polarizationAlice[i] == 'D':
            keyAlice += '0'
        elif polarizationAlice[i] == 'A':
            keyAlice += '1'
        else:
            keyAlice += '-'
print("keyAlice      = " + keyAlice)

keyAlice      = 101110100100100010000111001110000101110010111011000100110111010111111000001011000101101010010100100001011110101011011010001111011111111110000001100000101110110000000110110010110110010101000111000000101001011101010010110100100010011010011111000100000110111000111101011110001011001110111001000101000001101101101000011100101110101100100101000101110111100110111101000111101001111000001001100100100101011000011100101011010000010101000000110110110010011110100110110011000101000011011110111101100101000011001001111010111000110011000101001001011110010110011111111010001100011100011000010101100110100010110000001101101001111011110000011001010000011011011011010100101110001010000010001111011111000011000101110011100000101101101010111000001100101000011000111001101110000001000111111010111001110001111100001000011100110100100100010010011001110111010100101100101100000000000000101001001011001001110011011100000010100000101101001101000010011111010010000011110000111000111001111011111000010101110100

In [ ]:
# Don't think this is needed

# # Alice selects a photon state according to the key and basis.
# # This should be a string of the characters 'H', 'V', 'D', 'A'.
# photonAlice = ""
# # TODO: Put your code here.
# for index in range(n):
#   if basisAlice[index] == "+":
#       if keyAlice[index] == "0":
#         photonAlice += "H"
#       elif keyAlice[index] == "1":
#         photonAlice += "V"
#   elif basisAlice[index] == "x":
#       if keyAlice[index] == "0":
#         photonAlice += "D"
#       elif keyAlice[index] == "1":
#         photonAlice += "A"
# print("photonAlice  = " + photonAlice)

In [ ]:
# Eve selects a subsample of photons from Alice to measure.
# interceptIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=intercepted
interceptIndex = ""
# TODO: Put your code here.
for i in range(n):
  if random.randint(0,1) == 0:
    interceptIndex += "0"
  else:
    interceptIndex += "1"
print(interceptIndex)

0010110001001101011101010001110101111001011001010110110100110110100001000110110001101000100010101010001101100000110100011011000010010000111111001011010100101010000111010011101010011001101000110111100111110010101100010010111110001110111010011001111111011110001000001100000111011100001111111010010101011110010111000010100110110100010100110101100101110100011001001000100010001101001001101011101110000010011111100011001010111111010110000011100010101000111010111110100110111111101001011100000011000000111110000000100001000001010100101001001001001101011000100111001000010111010100010101100000011010101110001010010110101001100010101111111011101001101100000100000101100111011000000101110011101010010001001001001100110010001011001100000101110111110100110110100011010000100000100111100100111101100111010110000110010100110000110111101110100100010111111110110011101110000110101011110111011000111100100111001011111011111110011010111001000111111101000110000001111100111011100100000111101010110011011110111110000001

In [ ]:
# Eve chooses a basis to measure each intercepted photon.
# basisEve should be a string of n characters.
# Use the convention '+'=H/V, 'x'=D/A, ' '=not measured
basisEve = ""
# TODO: Put your code here.
for i in range(n):
  if interceptIndex[i] == '1':
    if random.randint(0,1) == 0:
      basisEve += "+"
    else:
      basisEve += "x"
  else:
    basisEve += ' '
print(basisEve)

  x x+   +  ++ + +xx + x   xx+ x +xxx  x +x  x + x+ xx x  ++ x+ x    x   x+ xx   +x +   x   x x x x   +x +x     +x +   ++ x+    +  x    ++x++x  + xx x x  x x +    +xx +  +xx x +  xx  xx x   xx +++x  +xxxx  x x +x   x  + x+x++   ++x ++x +  xx  x++xx+x x++x   +     x+     ++x xx+    +++x+x+ x  x x x x+xx  x +x+    x x  +x xx x   x x  +x x +x  x +++ x   x+  +  x   +   x   ++ x  x  +x + +++ +xx     +  x+xxx+   ++  x x ++++xx x xx     xx+   x + +   xxx x xx++x x  ++ xx++x++ +  x +xx      xx      x++++       +    +     + + +  + +  +  x  x  ++ x +x   x  xxx  +    x xx+ + +   + + x+      +x x x x+x   + +  + x+ x +  +x   + x xxx+x+x xx+ x  x+ ++     +     x ++  +++ ++      + x+x  +xx x x  x   x  +  +  +x  x+  +   + x+  x+     x +x+ +xx++ +  x+ xx x   +x x    +     x  +x+x  x  ++xx x+  x++ + ++    xx  + +  x+    xx +++x xxx x  x   + x+xxxxx+ ++  xx+ x+x    +x + + xxxx +x+ ++   +x++  +  ++x  + +xx+x +x++++x  ++ x xx+  x   +x+xx+x x   +x      ++xxx  ++x xxx  +     ++xx x x +x  ++ ++xx xxxx+      +

In [ ]:
# Eve performs a measurement on each photon.
# outcomeEve should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeEve = ""
# TODO: Put your code here.
for i in range(n):
    if basisEve[i] == '+':
        outcomeEve += QC.measureHV(qcArray[i], .5)
    elif basisEve[i] == 'x':
        outcomeEve += QC.measureDA(qcArray[i], .5)
    else:
      outcomeEve += ' '
print(outcomeEve)

  D NN   V  VN N NDA H M   NNV M MNNN  D NN  N H DV AN A  HN AN N    A   NN ND   NM V   N   N N D D   NA HM     ND N   NH NV    H  N    HVNNMD  N AD M N  N N N    VNM H  MND D N  AN  NN N   AN HNNN  NANNN  D M VN   N  N DNNVN   NNN NNN M  DN  NHVNDVD AHMN   N     NN     HNA DNN    VNNAMNN N  N N N DNAA  M NNH    N N  VN ND A   A N  NN A NA  D VNN A   NN  N  N   N   N   HN A  N  ND H NHM NNA     H  NNNAAV   HN  M N NNNHND D NN     NDN   N N N   DND N NNNNA N  NN NNHNNVN H  N NDN      NN      DMVNN       N    N     M H H  H V  N  A  A  NN A NN   N  ADN  N    D NNN N H   N V NV      MD N N NNA   N N  N AH D N  ND   N A NNMNNHN NNN A  DH NV     V     M NN  NMN NV      H ANN  HNA N N  N   A  M  N  VD  MN  V   V AN  AV     N NNV NNDNN V  NH ND N   NN N    H     N  NDVN  A  NHNN DN  NMN N VN    AN  N V  NN    NN NNVA NAA N  N   N AMMDNMNN MN  NNV DND    HA V H ANDA VDN HN   NNHN  N  HNA  N HANNN HDVVHHN  HN A MAN  N   VNHADHD N   VD      NNNDA  NVA NDA  N     MHNM D N NN  VM HNNM ADNMN      V

In [ ]:
# Eve resends photons to Bob.
# Be sure to handle the cases in which Eve gets an invalid measurement.
# TODO: Put your code here.
for i in range(n):
  if outcomeEve[i] == "H":
    QC.eveSendH(qcArray[i])
  elif outcomeEve[i] == "V":
    QC.eveSendV(qcArray[i])
  elif outcomeEve[i] == "D":
    QC.eveSendD(qcArray[i])
  elif outcomeEve[i] == "A":
    QC.eveSendA(qcArray[i])
  elif outcomeEve[i] == "N" or outcomeEve[i] == 'M':
    chooseRand = random.randint(0, 3)
    if chooseRand == 0:
      QC.eveSendH(qcArray[i])
    elif chooseRand == 1:
      QC.eveSendV(qcArray[i])
    elif chooseRand == 2:
      QC.eveSendD(qcArray[i])
    elif chooseRand == 3:
      QC.eveSendA(qcArray[i])
    #send random if N also choose based off of outcome if blank no change

In [ ]:
# Bob   --------------------------------------------

# Bob chooses a basis to measure each photon.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisBob = ""
for index in range(n):
  if random.randint(0,1) == 0:
    basisBob += "+"
  else:
    basisBob += "x"

print("basisBob    = " + basisBob)

basisBob    = ++xx++++x+xx++x+++x++++x+x++++++xx+xxxxxxx++++x+xxxxx++xx+x+x++x++x+++++++xx+++x+++xxxxxx+++xxx+++++xx+x+++xx+xx++xx+xx+x++xxxx++xxx++xx+xxxxxxxxx+++xxxxx+x+++++++xxxx++xx++x++++xx+xx+xx+xx++xx+++x++xx+x++xxxxxx+x+xx+++xxxxx+++xxxx+x++x+++xxx++xxxxxxx+x+xx+x++++x++xx++++xxxx+x+++x+x+x+xx++x+++xx+++++xx+x+xx++++x+x+++xxxxxx++++++x++++x+x+xx+xxx+xxxxxx+xxxx+x+x++xxx+xx++++++xx+++x+xxx+x+xx+xxxxx+++xxx+xxxx+++xx+++x+x+x++x+++++xx+++++xxx+xxxxxx+xx+xx++x+xxx+xxxxx++x+x+xxx+x+xxx+xxx+xx+x+++xx+x++x+x++x+x+xxxx++xx+x+x++++++++xx+xx+x+xx+++xxx+xxxx++++x+x++xx++x+xxx++x+xxxx+xx+x++x+xxxx+xx++xxxx++x+x+xx+xxx+xxxx++xx+x+x+x++x+++x++++x++x+xx+++++xx++++xxxx+++++x+xxxxx+x+x+x+xx+xx+xx++x++++x+++x++x+x++++xx++x+x++xx+xxx+x++x+++++xx+x+++x++xxxxxx++x+xx+x++x++xxxx++++x+x++++x+xx+x++x+xx+xxx++xxxxxx+++x++x++xx+++xxxxxxx++xx+x+++x++++x++++++x++++xx+x+x+xx++x+xx+xx+xx+xx+xx+x++++x+++++++xx+xx+xx+xxx++xxxx++x+x++xxx+x+x+xxxx+x++x++x+x+x++++xxxx++xx+xxxx+xx+x++++++x+x+++xx+xxx+xx++++x+xxxx

In [ ]:
# Bob performs a measurement on each photon.
# Use the methods of the Photon class to measure each photon.
# outcomeBob should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeBob = ""
for i in range(n):
    if basisBob[i] == '+':
        outcomeBob += QC.measureHV(qcArray[i], .5)
    elif basisBob[i] == 'x':
        outcomeBob += QC.measureDA(qcArray[i], .5)
print("outcomeBob  = " + outcomeBob)

outcomeBob  = NMDNHNHHAVNNVHANHNDNNHNDNANHVVHVNDVNNADDNNNVNVNHADNNAHNANHNNNNNNVNMNNNNNNVNNNNHAHNNNNNNNDNVVNNNNNNNVANHAHHVNNMDDNNNAHANVNVHNAANHHNNDNHNNHNDNNDNDANNNVNNNAMHNHHNVNHVNDDDHMNANNANVNNNANAANDNHNANNNNHNHNVNNAHDVNDDNNNNNNVADNVHADANNNMNNANNNDNNNVNNDDNNHNNDDNDDNNNDDHAVNNVDNHNNNVMHNNANNDVVVMNNNDNNNHVDNNNMDHVMNNAAMNHDNNHMHANAHVMANNNDDVNVVMNAHNNHNHAVAAHDDNVNNDANMNANNNNDNDNNNNAMDAHNNHVVAANNNNVDNNNDHDNVAAANNNVHNNANNAANVHHNANHNNVMHDNHNNNNNNNNNNVHNDANHNNNNMNNNANNDVNNVNADNAAMNNNHNVNNNNANNVNNDNDDNNANVNVVVNNNNNNDVNVVNNNMANNNHNADNNNDHVNHNHNHNDVNDVAVAANNNNANVANNAHNVHNHANNNNVHDVNDNNNANNANDNNNHNHHNHADMDHADNVANDNNNMNNVNDNNNNNNNDNVNNNNANNNDNHAHHNNHHMNDVVAVNDHMNVVAAHHVVAANDNVHVHDNNNDNNNAMNMMHNANDDVNDNVNNNNHANHVANNNVANVNVDNNNNVNVHNAVNAANNNVMNNHNHDDVNNNHDVNNNANNNNVDMNNNNNVANNANNNVVHHMVDVVNVNNDANMNHDNADVDNNNNADNNDNHMNANHMNNNNNVNNNNAANNNVNAVNNNVNNNVMNNHHNNNDNNNNANNDNDVNNNNDVNDMNAMNAHDANDAVNNHHHNNNNHNHVNNNADHDANMNNHNNANNHNNVNHNDNNHANMHAANNVNNHNVHANNNDNNNMNDDNNNAANNADAHNDVAMNNNNVAVNNNNDNNNADNDNVVNNNNNNNA

In [ ]:
# Bob infers the raw key.
# keyBob should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement

# Modified to accomodate Psi-
keyBob = ""
for i in range(n):
    if basisBob[i] == '+':
        if outcomeBob[i] == 'H':
            keyBob += '1'
        elif outcomeBob[i] == 'V':
            keyBob += '0'
        else:
            keyBob += '-'
    elif basisBob[i] == 'x':
        if outcomeBob[i] == 'D':
            keyBob += '1'
        elif outcomeBob[i] == 'A':
            keyBob += '0'
        else:
            keyBob += '-'
print("keyBob      = " + keyBob)

keyBob      = --1-1-1100--010-1-1--1-1-0-10010-10--011---0-0-101--01-0-1------0--------0----101-------1-00-------00-10110---11---010-0-01-00-11--1-1--1-1--1-10---0---0-1-11-0-10-1111--0--0-0---0-00-1-1-0----1-1-0--0110-11------001-01010------0---1---0--11--1--11-11---11100--01-1---0-1--0--1000----1---101----110---00--11--1-10-010-0---110-00--01--1-10000111-0--10---0----1-1----0-101--10000----01---111-0000---01--0--00-011-0-1--0-11-1----------01-10-1--------0--10--0-01-00----1-0----0--0--1-11--0-0-000------10-00----0---1-01---110-1-1-1-10-100000----0-00--01-01-10----0110-1---0--0-1---1-11-101-1101-00-1------0-1-------1-0----0---1-1011--11--10000-11--0000110000-1-01011---1---0----1-0-110-1-0----10-100---00-0-01----0-01-00-00---0---1-1110---110---0----01------00--0---0011-0100-0--10---11-0101----01--1-1--0-1------0----00---0-00---0---0---11---1----0--1-10----10-1--0--0110-100--111----1-10---01110----1--0--1--0-1-1--10--100--0--1-010---1-----11---00--0101-100-----000----1---01-1-00-------0

In [ ]:
# -----------------------------------------------------------
# Alice and Bob now publicly announce which bases they chose.
# Bob also announces which of his measurements were invalid.
# -----------------------------------------------------------

In [ ]:
# Alice & Bob ----------------------------------------------------------

# Alice and Bob extract their sifted keys.
# siftedAlice and siftedBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
siftedAlice = ""
siftedBob   = ""
# TODO: Put your code here.
for index in range(n):
    if keyAlice[index] == '-' or keyBob[index] == '-':
        siftedAlice += ' '
        siftedBob += ' '
    elif basisAlice[index] != basisBob[index]:
      siftedAlice += ' '
      siftedBob += ' '
    else:
        siftedAlice += keyAlice[index]
        siftedBob += keyBob[index]
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)

siftedAlice =   1 1 1 0   100 1    1 1    10    0   00     0      0  1 1               0    0         1  1        0   1 1    0     0 0     1  1  1      0    11   0       1  0 00  1    0  0 1   0 1  0 0 0    0 0      0  1       01  1010           1   1   0  1          10  1   0     1 0                 000          0    1  0      0 1     1      0    00010  1              0        010    10      0     0 100        0  1  0     1    00             1    1        1      1 1   1    1 1          1  1    1   0      10 10        1  0   100 1   1    1  101       1  01 1  1       1      1  0     0  1 1 0  10  00 0                  1        0   1   1   0         11 1 01 1    1  0               1 10  1 1       00    10 1       1    1   0       0        1 0   1    1                0   1 1  0       11 0001     0  1      1           10   0 10   0                 1  1          0          0 0   11      1     01 1       0  0  0   1      0 0     0       0           0  011  001      1     1   0             0

In [ ]:
# Alice and Bob use a portion of their sifted keys to estimate the quantum bit error rate (QBER).
# sampleIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=sampled
# The QBER is the fraction of mismatches within the sampled portion.
# For large samples, it should be close to the actual QBER,
# which Alice and Bob, of course, do not know.
sampleIndex = ""
sampledBobQBER = 0
# TODO: Put your code here.
numError = 0
numTotal = 0

# out of non-removed sifted key, 50/50 chance to sample it
for i in range(n):
    if siftedAlice[i] == ' ' or random.randint(0, 1) == 0:
        sampleIndex += '0'
    else:
        sampleIndex += '1'
        numTotal += 1

#if sampling and it does not match add one to the number of errors
for i in range(n):
    if sampleIndex[i] == '1' and siftedAlice[i] != siftedBob[i]:
       numError += 1

sampledBobQBER = numError / numTotal
print(str(sampledBobQBER) + " QBER")
print("sampleIndex = "+sampleIndex)

0.5042735042735043 QBER
sampleIndex = 00000000000001001000000000001000000000010000010000001000010000000000000000000010000000000001000000001000001000010000000000000100000000000010000110001000000010010110010000000100000000001000100000000000001000000000011000010000000000000000000010000000000000110010001000000000000000000000000010100000000001000000000000000010000000000001000000100001000000000000000000000001110000100000001000001011000000000100000000000100001000000000000001000000000000000000000010001000000000000000000001000010001000000000100000000010000000010000010000000000000000010011010000000000100000000010000010000100000100100100000000000000000000000000000001000100000000000000001001000000000100000000000000010110000000000000100001100000000010000000010000000100000000100000100001000000000000000000001000010000000100110100000000000000000000000000001000101000000000000000000000010010000000000100000000001010001100000010000001000000000000000100010000000010000000000000000000000000000011001010000000

In [ ]:
# Alice and Bob remove the portion of their sifted keys that was sampled.
# Since a portion of the sifted key was publicly revealed, it cannot be used.
# secureAlice and secureBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
secureAlice = ""
secureBob = ""
# TODO: Put your code here.
for i in range (n):
  # if we sample it is removed
  if sampleIndex[i] == '0':
    secureAlice += ' '
    secureBob += ' '
  #if we don't sample we just pass it on
  else:
    secureAlice += siftedAlice[i]
    secureBob += siftedBob[i]
print("secureAlice = " +secureAlice)
print("secureBob =   "  + secureBob)

secureAlice =              0  1           1          0     0      0    1                    0            1        0     1    0             1            0    11   0       1  0 00  1       0          0   0             0          01    1                    0             10  1   0                         0 0          0                1            0      0    1                       010    1       0     0 10         0           1    0              1                      1   1                    1    1   0         1         1        0     1                 1  01 1          1         0     0    1     0  0  0                               1   1                1  1         0               1 10             0    10         1        0       0        1     1    1                    1    0       1  00 1                            0   0 1                      1  1          0          0 0   11      1      1               0   1        0                             11  0 1                0              

In [ ]:
# Alice and Bob make a hard determination whether the channel is secure.
# If it looks like there's something fishy, better hit the kill switch!
channelSecure = True # default value, to be changed to False if Eve suspected
# TODO: Put your code here.
if (sampledBobQBER > .15):
  channelSecure = False

In [ ]:
# Eve infers the raw key.
# keyEve should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement, ' '=not measured

# Also adjusted for Psi-
keyEve = ""
for index in range(len(outcomeEve)):
  if outcomeEve[index] == "H":
    keyEve += "1"
  elif outcomeEve[index] == "V":
    keyEve += "0"
  elif outcomeEve[index] == "D":
    keyEve += "1"
  elif outcomeEve[index] == "A":
    keyEve += "0"
  elif outcomeEve[index] == "N" or outcomeEve[index] == "M":
    keyEve += "-"
  else:
    keyEve += " "

print("keyEve     = " + keyEve)

keyEve     =   1 --   0  0- - -10 1 -   --0 - ----  1 --  - 1 10 0- 0  1- 0- -    0   -- -1   -- 0   -   - - 1 1   -0 1-     -1 -   -1 -0    1  -    10---1  - 01 - -  - - -    0-- 1  --1 1 -  0-  -- -   0- 1---  -0---  1 - 0-   -  - 1--0-   --- --- -  1-  -10-101 01--   -     --     1-0 1--    0--0--- -  - - - 1-00  - --1    - -  0- -1 0   0 -  -- 0 -0  1 0-- 0   --  -  -   -   -   1- 0  -  -1 1 -1- --0     1  ---000   1-  - - ---1-1 1 --     -1-   - - -   1-1 - ----0 -  -- --1--0- 1  - -1-      --      1-0--       -    -     - 1 1  1 0  -  0  0  -- 0 --   -  01-  -    1 --- - 1   - 0 -0      -1 - - --0   - -  - 01 1 -  -1   - 0 -----1- --- 0  11 -0     0     - --  --- -0      1 0--  1-0 - -  -   0  -  -  01  --  0   0 0-  00     - --0 --1-- 0  -1 -1 -   -- -    1     -  -10-  0  -1-- 1-  --- - 0-    0-  - 0  --    -- --00 -00 -  -   - 0--1---- --  --0 1-1    10 0 1 0-10 01- 1-   --1-  -  1-0  - 10--- 110011-  1- 0 -0-  -   0-10111 -   01      ---10  -00 -10  -     -1-- 1 - --  0- 1---

In [ ]:
# Eve extracts her sifted key.
# Knowing what Alice and Bob have publically revealed, Eve
# now selects which portion of her sifted key to keep.
# stolenEve should be strings of length n.
# Use the '0', '1', ' '=removed
stolenEve = ""
# TODO: Put your code here.
# i have keyalice and keybob
# i add
for i in range(n):
    if keyEve[i] == '-' or keyEve[i] == ' ' or basisAlice[i] != basisEve[i]:
        stolenEve += ' '
    else:
        stolenEve += keyAlice[i]
print("stolenEve = " + stolenEve)

stolenEve =   1         1      0 1       0         0            0  1  1  1                                  1                               1        0                         0               0             0                          0  0                        0  01                  0                             0       0                           0  0  1                               0      0         0           1     1            1 1                      1         0               0                     1 0                      1    0       0        1                           1     1  0                                   1            1             1     1                         1    1 1                                                            0                0          0       1   0                     1              1   1                         0 0      0        0  0   1      1      1 1     0    00001    1                  0                 11   01   0         1              0    

In [ ]:
# ANALYSIS -------------------------------------------------------------

# Below is a standard set of metrics to evaluate each protocol.
# You need not change any of what follows.

# Compare Alice and Bob's sifted keys.
numMatchBob = 0
actualBobQBER = 0
secureKeyRateBob = 0
secureKeyLengthBob = 0
for i in range(len(secureAlice)):
    if secureAlice[i] != ' ':
       secureKeyLengthBob += 1
       if secureAlice[i] == secureBob[i]:
           numMatchBob += 1

# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1-actualBobQBER) * secureKeyLengthBob / n * 1e6;

# Compare Alice and Eve's sifted keys.
numMatchEve = 0
actualEveQBER = 0
stolenKeyRateEve = 0
stolenKeyLengthEve = 0
for i in range(len(stolenEve)):
    if stolenEve[i] != ' ':
       stolenKeyLengthEve += 1
       if secureAlice[i] == stolenEve[i]:
           numMatchEve += 1
# Compute the actual quantum bit error rate for Eve.
if stolenKeyLengthEve > 0:
    actualEveQBER = 1 - numMatchEve / stolenKeyLengthEve
else:
    actualEveQBER = float('nan')
# Compute the stolen key rate, assuming each trial takes 1 microsecond.
stolenKeyRateEve = (1-actualEveQBER) * stolenKeyLengthEve / n * 1e6;
#to do
# bell inequality
# finish report
#collect data
# for fun do immage encruption or etc

# DISPLAY RESULTS ------------------------------------------------------

print("")
print("basisAlice  = " + basisAlice)
print("basisBob    = " + basisBob)
print("basisEve    = " + basisEve)
print("")
print("keyAlice    = " + keyAlice)
print("keyBob      = " + keyBob)
print("keyEve      = " + keyEve)
print("")
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)
print("")
print("secureAlice = " + secureAlice)
print("secureBob   = " + secureBob)
print("stolenEve   = " + stolenEve)
print("")
if not channelSecure:
    secureKeyRateBob = 0;
    stolenKeyRateEve = 0;
    print("*********************************************")
    print("* ALERT! The quantum channel is not secure. *")
    print("*********************************************")
    print("")
print("sampledBobQBER = " + str(sampledBobQBER))
print("actualBobQBER  = " + str(actualBobQBER))
print("actualEveQBER  = " + str(actualEveQBER))
print("")
print("secureKeyRateBob = " + str(secureKeyRateBob/1000) + " kbps")
print("stolenKeyRateEve = " + str(stolenKeyRateEve/1000) + " kbps")

# Your goal is to maximize secureKeyRateBob and minimize stolenKeyRateEve.


basisAlice  = xxx++++xxx+x++xx+++xx++x++xx++xx+++xx+xx+++xx+xx++x+xx+xx++x+x+xxxxx++xxx+++++++x++xxx++xxx++xx+x++xxxx++x++xx+xx+++xx++xxxx+x+x+x+xxx++x+xx+++xxx+++xxx++xx+xx++++++x+x+xxx+x+++x+xxx++xx++xx+xx+x++x+++xxxxx+xx+xxx+x++++xx+++xxx++x+xxx+x++x+x+x+xx+++++x+xxxx++x+xxxx+x++x+++++++xxx++xx++xx++x+++x+xx+++x+xxxx++++x+x+x++xxx+++++xxx++++xx++x+xxx+xxxxx++++x+x+xxxx+xxx+++xx+x+xx+x+++xxxx+x++xx++xx++++xxxxx++x+x+xx++x+++xx+x+xxxxx++x+x+x++++++x++xx+x+xx++xxx++x+x+xx+xx+x+xxxx+++xx+xx+xx+++++xx+x++xxxx+++++++++xxx+++xxxxx++x+xxx+++x+xx++xx+++x++xx+xx+x+x+++++++xxxxx++++xx+x++xxx++x++++x+++x+++x+xx++xxxxx++xxx+xx++++x+x+x+xx+x++x+x+x++xxx+x++x+++++xx++xx+x+x+xx++x+++xxx+x+xxxxx+xxx+xx++xx+x++++++x++x+++x++x+++xxx+xx++x+x+xxxx+xx++xxx++++++xx+x+++++x++++x++x++x+x+xxx+++x+++x++++x+x+xx+xx+x++xxxxxxx++x+x+x+++x+x+xxxxxx+xx+x+x++x+xxxxxx+xx++++xxx+x++x++x++x+xx+++x+x+++x++++x++x+xxx++xx+x+x++x++xxxx+x+x+x+++xxxx+x++x++xx+x+x+xxx++++x++x+x++++++x++xxxxxx+x+x+x+xxx+xx+x++xx++++xxx+xx+++

In [ ]:
# bellTestIndex = ""
# for i in range(n):
#     if random.randint(0, 1) == 0:
#         bellTestIndex += "0"
#     else:
#         bellTestIndex += "1"

# bellAlice = ""
# bellBob = ""
# for i in range(n):
#     if bellTestIndex[i] == "1":
#         bellAlice += siftedAlice[i]
#         bellBob += siftedBob[i]

# def chsh_inequality(alice_results, bob_results):
#     # Convert results from string to integers, filtering out any spaces
#     alice_results = [int(x) for x in alice_results if x != ' ']
#     bob_results = [int(x) for x in bob_results if x != ' ']

#     if len(alice_results) != len(bob_results):
#         raise ValueError("Alice and Bob results must have the same length.")

#     chsh_value = 0
#     settings = [(0, 0), (0, 1), (1, 0), (1, 1)]  # Example settings
#     for i in range(len(alice_results)):
#         alice_setting, bob_setting = settings[i % 4]
#         chsh_term = (-1) ** (alice_results[i] + bob_results[i] + alice_setting * bob_setting)
#         if (alice_setting == 0 and bob_setting == 0) or (alice_setting == 0 and bob_setting == 1):
#             chsh_value += chsh_term
#         else:
#             chsh_value -= chsh_term

#     return abs(chsh_value) / len(alice_results)

# # Example usage:
# bellAlice = '0110 101'  # Simulated binary string for Alice
# bellBob = '1011 010'    # Simulated binary string for Bob

# # Convert strings to lists and remove spaces
# bellAlice = [x for x in bellAlice if x != ' ']
# bellBob = [x for x in bellBob if x != ' ']

# # Calculate CHSH value
# chsh_value = chsh_inequality(bellAlice, bellBob)
# if chsh_value > 2:
#     print("Bell's inequality violated. Eavesdropper detected!")
# else:
#     print("No eavesdropper detected.")
